<a href="https://colab.research.google.com/github/Untick/NetOptik/blob/Makarikova-Olga-branch/Makarikova_NetOptik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Базовые команды

In [ ]:
!pwd()

'/content'

## Импорт необходимых библиотек

In [ ]:
!pip install lxml

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=9eb3c7af4b89b0fe7950bd4dad4cda853b4e3555b5ad80ca49019622b88691aa
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from bs4 import BeautifulSoup
import lxml
import requests
#import wget

## Парсинг сайта Netoptik

Загрузим саму страницу по адресу https://www.netoptik.ru/yandexmarket.yml к себе на компьютер и откроем в BS

In [ ]:
# парсинг кода страницы
content = []
with open('netoptik.ru_yandexmarket.yml', 'r') as f:
  content = f.readlines()
content = "".join(content)
bs = BeautifulSoup(content, "lxml")

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Найдем ссылки на изображения

In [ ]:
# <picture>
allimgs = bs.find_all('picture')
allimgs

In [ ]:
allimgs[0].text

'https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s213-600x600.jpg'

Сохраним все изображения

In [ ]:
url = "https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s213-600x600.jpg"
#wget.download(url)

'balenciagabal01088o0s213-600x600 (1).jpg'

In [ ]:
# номер изображения
num = 1

# проходим по всем объектам img
for img in allimgs[:]:

  # достаем ссылку на изображение
  img_url = img.text
  print(img_url)

  # get запрос
  filename = requests.download(img_url)
#  print(filename.text)

  # сохраняем изображение в файл
  with open(f'img_{num}.jpg', "wb") as out:
    out.write(filename.content)

  num+=1

Ни wget, ни requests не может загрузить картинки с сайта, так как сервер запрашивает прохождение капчи. Воспользуемся уже скачанными изображениями.

Достанем характеристики очков из файла netoptik.ru_yandexmarket.yml

In [ ]:
offers = bs.find("offer")

In [ ]:
offers

<offer id="11365">
<categoryid>176</categoryid>
<name>Оправа Balenciaga, BAL 0108, 8O0</name>
<url>https://www.netoptik.ru/frames/oprava-balenciaga-bal-0108-8o0</url>
<price>6650</price>
<oldprice>9500</oldprice>
<currencyid>RUB</currencyid>
<vendor>BALENCIAGA</vendor>
<model>Оправа Balenciaga, BAL 0108, 8O0</model>
<picture>https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s213-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s231-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s222-600x600.jpg</picture>
<param name="Категория"/>Оправы для очков
<param name="Ширина оправы (мм)"/>128
<param name="Ширина линзы (мм)"/>55
<param name="Высота линзы (мм)"/>40
<param name="Мост (мм)"/>16
<param name="Длина заушника (мм)"/>140
<param name="Материал "/>комбинированный
<param name="Стиль"/>Унисекс
<param name="Строение оправы"/>Ободк

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
offers.find("param", {"name":"Материал "})

<param name="Материал "/>

Скачаем изображения для каждого оффера в разные папки согласно их материалу

In [ ]:
c = 1 # номер оффера

for offer in offers[:2]:
  try:
      folder = offer.find("param", {"name":"Материал "}).text # разделим изображения по папкам по материалу
  except:
      continue
  print(folder)
  imgs = offer.find_all("picture") # найдем и скачаем все картинки данного оффера
  num = 1 # номер изображения
  for img in imgs:
      img_url = img.text # достаем ссылку на изображение
      filename = requests.get(img_url) # скачиваем картинку
      with open(f'{folder}/img_{c}_{num}.jpg', "wb") as out: # сохраняем изображение в файл
          out.write(filename.content)
      num += 1
  c += 1

TypeError: ignored

Скачаем изображения для каждого оффера в разные папки согласно их дужкам

In [ ]:
c = 1 # номер оффера

for offer in offers[:2]:
  try:
      folder = offer.find("param", {"name":"Строение оправы"}).text # разделим изображения по папкам по типу дужек
  except:
      continue
  imgs = offer.find_all("picture") # найдем и скачаем все картинки данного оффера
  num = 1 # номер изображения
  for img in imgs:
      img_url = img.text # достаем ссылку на изображение
      filename = requests.get(img_url) # скачиваем картинку
      with open(f'{folder}/img_{c}_{num}.jpg', "wb") as out: # сохраняем изображение в файл
          out.write(filename.content)
      num += 1
  c += 1